In [0]:
import mlflow
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [0]:
# --- PASSO A: Configurar o Experimento ---
# Isso garante que o MLflow grave tudo associado a este notebook
mlflow.autolog()

print("Autolog ativado.")

# --- PASSO B: Gerar dados fake (Dummy Data) ---
# Só para garantir que o erro não seja nos dados
print("Gerando dados sintéticos...")
X, y = make_classification(n_samples=100, n_features=5, random_state=42)

# --- PASSO C: Treinar o Modelo de Teste ---
print("Iniciando treino de teste...")

# Damos um nome para a 'run' para achar fácil depois
with mlflow.start_run(run_name="Teste_Infra_SquadB"):
    # Criamos um modelo simples
    clf = RandomForestClassifier(n_estimators=10, max_depth=3)
    
    # O .fit() aqui vai disparar o MLflow automaticamente!
    clf.fit(X, y)
    
    # Vamos logar uma métrica fake só de garantia
    mlflow.log_metric("recall_teste", 0.85)

print("✅ Sucesso! O modelo foi treinado e logado.")

In [0]:
import mlflow.deployments

client = mlflow.deployments.get_deploy_client("databricks")

# Lista de tentativas (do mais moderno pro mais antigo)
candidatos = [
    "databricks-meta-llama-3-1-70b-instruct", # O que falhou antes
    "databricks-meta-llama-3-70b-instruct",   # Llama 3.0 (Muito provável funcionar)
    "databricks-dbrx-instruct",               # DBRX (Modelo da Databricks)
    "databricks-mixtral-8x7b-instruct"        # Mixtral (Leve e rápido)
]

print("🔍 Iniciando varredura de modelos disponíveis...\n")

modelo_encontrado = None

for modelo in candidatos:
    print(f"👉 Tentando conectar em: {modelo}...")
    try:
        client.predict(
            endpoint=modelo,
            inputs={"messages": [{"role": "user", "content": "Teste"}]}
        )
        print(f"✅ SUCESSO! O modelo '{modelo}' está ativo e funcionando!")
        modelo_encontrado = modelo
        break # Para na primeira vitória
    except Exception as e:
        if "ENDPOINT_NOT_FOUND" in str(e) or "404" in str(e):
            print(f"❌ Não encontrado (404).")
        else:
            print(f"⚠️ Erro diferente: {str(e)[0:100]}...")

print("\n------------------------------------------------")

if modelo_encontrado:
    print(f"🎉 MEMORIZE ISSO: Amanhã usem o nome: '{modelo_encontrado}'")
else:
    print("😓 Nenhuma das tentativas padrão funcionou. Listando TUDO o que existe no workspace:")
    try:
        endpoints = client.list_endpoints()
        # Filtra apenas os endpoints de sistema (Foundation Models)
        for ep in endpoints:
            if ep.creator == "system": 
                print(f"📌 Disponível: {ep.name}")
    except Exception as e:
        print(f"Erro ao listar endpoints: {e}")